## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
695,695,Omboue,GA,-1.5746,9.2618,76.86,90,100,10.76,light rain
696,696,Aguas Vermelhas,BR,-15.7472,-41.4600,77.04,33,0,4.83,clear sky
697,697,Qiongshan,CN,20.0058,110.3542,75.20,94,0,6.71,clear sky
698,698,Goya,AR,-29.1400,-59.2626,70.39,63,28,7.65,scattered clouds
699,699,Vrangel,RU,42.7297,133.0839,36.45,99,89,4.61,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Uberlandia,BR,-18.9186,-48.2772,86.00,37,40,9.22,scattered clouds
5,5,Arlit,NE,18.7369,7.3853,71.96,12,0,12.06,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,81.05,72,100,17.16,overcast clouds
10,10,Bengkulu,ID,-3.8004,102.2655,76.55,91,100,2.26,overcast clouds
13,13,Butaritari,KI,3.0707,172.7902,81.63,74,82,12.68,light rain
14,14,Bom Jesus,BR,-9.0744,-44.3586,83.66,44,100,2.28,overcast clouds
15,15,Avarua,CK,-21.2078,-159.7750,84.20,79,75,19.57,broken clouds
16,16,Cabedelo,BR,-6.9811,-34.8339,78.80,94,75,5.75,broken clouds
20,20,May Pen,JM,17.9645,-77.2451,86.00,65,20,28.77,few clouds
27,27,Hithadhoo,MV,-0.6000,73.0833,80.04,82,100,31.34,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Uberlandia,BR,86.00,scattered clouds,-18.9186,-48.2772,
5,Arlit,NE,71.96,clear sky,18.7369,7.3853,
7,Atuona,PF,81.05,overcast clouds,-9.8000,-139.0333,
10,Bengkulu,ID,76.55,overcast clouds,-3.8004,102.2655,
13,Butaritari,KI,81.63,light rain,3.0707,172.7902,
14,Bom Jesus,BR,83.66,overcast clouds,-9.0744,-44.3586,
15,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,
16,Cabedelo,BR,78.80,broken clouds,-6.9811,-34.8339,
20,May Pen,JM,86.00,few clouds,17.9645,-77.2451,
27,Hithadhoo,MV,80.04,heavy intensity rain,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Uberlandia,BR,86.00,scattered clouds,-18.9186,-48.2772,Hotel Apollo
5,Arlit,NE,71.96,clear sky,18.7369,7.3853,Case de Passage De la Prefecture
7,Atuona,PF,81.05,overcast clouds,-9.8000,-139.0333,Villa Enata
10,Bengkulu,ID,76.55,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Butaritari,KI,81.63,light rain,3.0707,172.7902,Isles Sunset Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))